In [ ]:
# Dependency imports
import re
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()
'''
sns.set_style({'axes.facecolor':'#E7E7F0', 'figure.facecolor':'white', \
               'axes.edgecolor': 'white', 'grid.color': 'white', \
              'axes.labelcolor': 'black', 'text.color': 'black', \
              'xtick.color': 'black', 'ytick.color': 'black'})
'''

# plotly standard imports
import plotly
plotly.tools.set_credentials_file(username='vivek.pandey', api_key='GcINHnvXXRBgEUUckTTf')

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 

# Cufflinks wrapper on plotly
import cufflinks

# Notebook customizations
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1
import warnings
warnings.filterwarnings('ignore')

sns.axes_style()

### Properties EDA

In [ ]:
properties_df = pd.read_csv("./output/engineered_trulia_properties.csv", sep="|", dtype={'zip_code': str}, encoding="latin1")
properties_df

In [ ]:
# Dropping columns that do not have any value for now
properties_df.drop(axis=0, inplace=True, columns=['basement_type', 'exterior_type', 'heating_type', \
                                                  'no_of_parking_spaces', 'no_of_stories', 'parking_type'])
properties_df

In [ ]:
properties_df.info()

### Analyze and fill missing data

In [ ]:
sns.heatmap(properties_df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

Lets look at how many rows have fully valid data
#### Right now, lets only keep entirely usable data and get to model training, we'll come back later for data imputation

In [ ]:
properties_df = properties_df[~(properties_df['no_of_bedrooms'].isnull() | properties_df['no_of_baths'].isnull() | \
              (properties_df['no_of_bedrooms'] == 0) | (properties_df['no_of_baths'] == 0) | \
              (properties_df['built_year'] < 1500) | (properties_df['tax_year'] < 1800) | \
              properties_df['tax_amount'].isnull() | (properties_df['tax_amount'] < 10) | \
                properties_df['area_sqft'].isnull() | (properties_df['area_sqft'] < 100) | \
              properties_df['lot_size'].isnull() | (properties_df['lot_size'] < 100))]
properties_df

#### We've dropped about half the records here, so data imputation is really important for the future to be able to use that data

In [ ]:
sns.heatmap(properties_df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

Lets look at the distribution of data in the columns to see come up with close values for imputation

In [ ]:
sns.boxplot(data=properties_df[~properties_df['no_of_bedrooms'].isnull() & (properties_df['no_of_bedrooms'] > 0)], \
            x='no_of_bedrooms', y='property_type')

In [ ]:
sns.pairplot(data=properties_df[['area_sqft', 'built_year', 'lot_size', 'no_of_baths', 'no_of_bedrooms', \
                                 'tax_amount', 'tax_year']])

### This subsection to prevent having to re-run the notebook from above

In [ ]:
properties_temp = properties_df

In [ ]:
properties_df = properties_temp

#### Find and remove outliers

In [ ]:
properties_q1 = properties_df.quantile(0.25)
properties_q3 = properties_df.quantile(0.75)
properties_iqr = properties_q3 - properties_q1

print (properties_q3)
print ('----------------')
print (properties_q1)
print ('----------------')
print (properties_iqr)

Some columns within a limited range might be better off without outlier filtering to prevent losing large amount of data, so remove the ones from the IQR series that we don't want to filter outliers for.

Also, adjust the weights to multiply the individual column IQR's by, to prevent overfiltering or underfiltering.
To gauge this, look at the pairplot above to get an idea of the data, and also the various quartiles calculated.

In [ ]:
properties_iqr = properties_iqr.drop(labels=['built_year', 'tax_year'])
properties_iqr

In [ ]:
prop_outlier_wts = {
    'area_sqft': 2,
    'lot_size': 10,
    'no_of_baths': 5,
    'no_of_bedrooms': 6,
    'tax_amount': 25
}

In [ ]:
for index, value in properties_iqr.items():
    print(index, value)
    # Select only the records that fall within the valid range, automatically ignores outliers
    properties_df = properties_df[(properties_df[index] >= (properties_q1[index] - prop_outlier_wts[index] * properties_iqr[index])) & \
                                  (properties_df[index] <= (properties_q3[index] + prop_outlier_wts[index] * properties_iqr[index]))]
    print(properties_df.shape)

In [ ]:
properties_df.reset_index(drop=True, inplace=True)
properties_df

In [ ]:
sns.heatmap(data=properties_df.corr(), linewidths=0.5, annot=True)

### High and medium correlated features from Properties

* area_sqft vs. no_of_baths (0.7)
* area_sqft vs. no_of_bedrooms (0.55)
* no_of_bedrooms vs no_of_baths (0.48)
* no_of_baths vs. tax_amount (0.46)
* area_sqft vs. tax_amount (0.46)

### Replotting relationships after removing outliers

In [ ]:
sns.pairplot(data=properties_df[['area_sqft', 'built_year', 'lot_size', 'no_of_baths', 'no_of_bedrooms', \
                                 'tax_amount', 'tax_year']])

### Plotting in US Map

In [ ]:
# Making a new column with a default value of one, so its value can be
# passed to the map
properties_df_for_map = properties_df
properties_df_for_map['z'] = 1

properties_map_data = dict(type='choropleth',
            colorscale = 'YlOrRd',
            locations = properties_df_for_map['state'],
            z = properties_df_for_map['z'],
            locationmode = 'USA-states',
            text = properties_df_for_map['state'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Properties in US"}
            ) 

In [ ]:
properties_map_layout = dict(title = 'US Properties by State',
                              geo = dict(scope='usa',
                                         showlakes = True,
                                         lakecolor = 'rgb(85,173,240)')
                             )

In [ ]:
properties_choromap = go.Figure(data=[properties_map_data], layout=properties_map_layout)

In [ ]:
'''
DO NOT UNCOMMENT, THE NOTEBOOK WILL FREEZE AND CRASH WHILE RENDERING THIS MAP
iplot(properties_choromap)
'''

### Transactions EDA

In [ ]:
transactions_df = pd.read_csv("./output/engineered_trulia_transactions.csv", sep="|", parse_dates=['recording_date', 'contract_date'], \
                              encoding="latin1")
transactions_df

In [ ]:
transactions_df.describe()

In [ ]:
sns.heatmap(transactions_df.isnull(), yticklabels=False, cbar=False, cmap='viridis')

#### Dropping the county_transfer_tax and total_transfer_tax columns cause they are heavily nulls and removing rows that have incomplete data cause they are few

In [ ]:
transactions_df = transactions_df.drop(columns=['county_transfer_tax', 'total_transfer_tax'], axis=1)
transactions_df

In [ ]:
transactions_df = transactions_df[~(transactions_df['contract_date'].isnull() | \
                                    transactions_df['document_type'].isnull() | \
                                   transactions_df['transaction_type'].isnull() | \
                                   (transactions_df['price'] < 1000))]
transactions_df

In [ ]:
transactions_df['recording_year'] = transactions_df['recording_date'].dt.year
transactions_df['recording_month'] = transactions_df['recording_date'].dt.month
transactions_df['recording_day'] = transactions_df['recording_date'].dt.day
transactions_df['contract_year'] = transactions_df['contract_date'].dt.year
transactions_df['contract_month'] = transactions_df['contract_date'].dt.month
transactions_df['contract_day'] = transactions_df['contract_date'].dt.day

transactions_df = transactions_df.drop(columns=['recording_date', 'contract_date'], axis=1)
transactions_df

In [ ]:
sns.boxplot(data=transactions_df, x='price', y='transaction_type')

In [ ]:
sns.pairplot(data=transactions_df)

### This subsection to prevent having to re-run the notebook from above

In [ ]:
transactions_temp = transactions_df

In [ ]:
transactions_df = transactions_temp

### Find, analyze and remove outliers in transaction data

In [ ]:
transactions_q1 = transactions_df.quantile(0.25)
transactions_q3 = transactions_df.quantile(0.75)
transactions_iqr = transactions_q3 - transactions_q1
transactions_iqr

In [ ]:
transactions_df[transactions_df['price'] > 20000000]

In [ ]:
transactions_df = transactions_df[transactions_df['price'] < 20000000]

In [ ]:
transactions_df[transactions_df['price'].isna()]

In [ ]:
sns.boxplot(data=transactions_df, x='price')

In [ ]:
# Filling in nans cause the plotting can't handle NaNs
transactions_df['price'].fillna(value=0, inplace=True)

In [ ]:
sns.boxplot(data=transactions_df, x='price')

In [ ]:
sns.distplot(transactions_df['price'], bins=5)

In [ ]:
sns.jointplot(data=transactions_df, x='total_transfer_tax', y='price', kind='reg')

In [ ]:
sns.boxplot(data=transactions_df, x='price')

In [ ]:
sns.heatmap(data=transactions_df.corr(), annot=True)

### High and medium correlated features from Transactions

* total_transfer_tax vs. county_transfer_tax (0.72)

In [ ]:
sns.pairplot(transactions_df)

In [ ]:
sns.relplot(data=transactions_df, x='price', y='total_transfer_tax')

### Playin around with plot.ly

In [ ]:
transactions_df[['price', 'total_transfer_tax']].iplot(kind='box')

In [ ]:
transactions_df['price'].iplot(kind='hist', xTitle='price', yTitle='count', title='Price Distribution')

In [ ]:
transactions_matrix_fig = ff.create_scatterplotmatrix(transactions_df[['price', 'total_transfer_tax', \
                                                                       'county_transfer_tax']], diag='histogram')
py.iplot(transactions_matrix_fig)

In [ ]:
transactions_df[transactions_df['price'] == transactions_df['price'].max()]

### Enriching transactions data with properties data

In [ ]:
transactions_en_df = pd.merge(transactions_df, properties_df, how='left', on=['property_url'])
transactions_en_df